In [59]:
#export
from export.nb_02 import *
from mimetypes import types_map
from collections import OrderedDict
from PIL import Image
from numpy import array

In [2]:
data_path=Path(r'd:\datasets\data\imagenette2-160')

data_path.ls()

['imagenette2-160.tar', 'train', 'val']

In [3]:
#export
def get_extensions(file_type):  return [i for i,j in types_map.items() if j.startswith(file_type)]

In [4]:
#export
image_extensions= get_extensions('image')

In [5]:
#export
def dir_files(p, fs, extensions=None):
    p=Path(p)
    files=[p/f for f in fs if not f.startswith('.')
          and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return files

In [6]:
#export
def get_files(root:Path, recurse=False, extensions=None):
    root= Path(root)
    if recurse:
        res=[]
        for p,d,f in os.walk(root):
            d[:] = [o for o in d if not o.startswith('.')]
            res+=dir_files(p, f, extensions)
        return res
    else:
        f=[o.name for o in os.scandir(root) if o.is_file()]
        return dir_files(root, f, extensions)

In [7]:
img_files=get_files(data_path, recurse=True, extensions=image_extensions)
img_class=[i.parent.name for i in img_files]

In [29]:
OrderedDict(map(reversed, enumerate(set(b))))

OrderedDict([('c', 0), ('a', 1), ('b', 2)])

In [33]:
#export
class Labeler():
    def __init__(self, y_train):
        self.label_dict=OrderedDict(map(reversed, enumerate(set(y_train))))
        
    def transform(self, ys):
        return [self.label_dict[i] for i in ys]

In [37]:
train_img=[i for i in img_files if i.parent.parent.name=='train']
val_img=[i for i in img_files if i.parent.parent.name!='train']
train_label=[i.parent.name for i in train_img]
val_label=[i.parent.name for i in val_img]

In [38]:
labels=Labeler(train_label)
y_train=labels.transform(train_label)
y_val=labels.transform(val_label)

In [39]:
assert len(y_train)==len(train_img)
assert len(y_val)==len(val_img)

In [40]:
len(y_val), len(val_img)

(3925, 3925)

In [90]:
torch.tensor(array(Image.open(train_img[0])), dtype=torch.uint8).shape

torch.Size([160, 213, 3])

In [80]:
#export
class Dataset():
    def __init__(self, x, y):
        self.x, self.y= x, y
        
    def __getitem__(self, i):
        return torch.tensor(array(Image.open(self.x[i])), dtype=torch.uint8), self.y[i]
    
    def __len__(self):
        return len(self.x)

In [81]:
train_ds, val_ds= Dataset(train_img, y_train), Dataset(val_img, y_val)

In [86]:
len(train_ds)

9469

In [50]:
train_dl, valid_dl= DataLoader(train_ds), DataLoader(val_ds)